In [ ]:
import torch
import resnet18 as r
from torchvision.datasets import MNIST
from torch.utils.data import Subset
import torchvision.transforms as transforms
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def getData(batch_size = 32):

    # Standard MNIST transform.
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    # Load MNIST train and test.
    ds_train = MNIST(root='./data', train=True, download=True, transform=transform)
    ds_test = MNIST(root='./data', train=False, download=True, transform=transform)

    # Split train into train and validation.
    val_size = 5000
    I = np.random.permutation(len(ds_train))
    ds_val = Subset(ds_train, I[:val_size])
    ds_train = Subset(ds_train, I[val_size:])

    train_dataloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True ,  num_workers=8)
    test_dataloader = torch.utils.data.DataLoader(ds_test, batch_size=batch_size,  num_workers=8)
    val_dataloader = torch.utils.data.DataLoader(ds_val, batch_size=batch_size,  num_workers=8)

    return train_dataloader, test_dataloader, val_dataloader

In [ ]:
resnet_ = r.ResNet18(1,10)

In [ ]:
from tqdm import tqdm


def model_pipeline():


    #make the model, data and optimization problem
    model, criterion, optimizer, trainloader, testloader, validationloader = create()

    #train the model
    train(model, trainloader, criterion, optimizer, validationloader)

    #test the model
    print("Accuracy test: ",test(model, testloader))
        
    #return model

def create():
    
    #Create a model
    model = resnet_.to(device)
    nparameters = sum(p.numel() for p in model.parameters())
    print(nparameters)
    #Create the loss and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    trainloader,testloader,validationloader = getData(batch_size=1024)

    return model, criterion, optimizer,trainloader, testloader, validationloader

# Function to train a model.
def train(model, trainloader, criterion, optimizer, validationloader):
 

    model.train()
    losses, valacc = [], []  

    for epoch in range(5):
        
        progress_bar = tqdm(trainloader, desc=f'Training epoch {epoch}', leave=False)
        
        for batch, (images, labels) in enumerate(progress_bar):
        
            loss = train_batch(images, labels,model, optimizer, criterion)
            acc = test(model, validationloader)
            progress_bar.update(1)
            
            logs = {"loss": loss.detach().item(), "val_acc": acc}
            progress_bar.set_postfix(**logs)
        
            losses.append(loss.item())
            valacc.append(acc)

    return np.mean(losses)

def train_batch(images, labels, model, optimizer, criterion):

    #insert data into cuda if available
    images,labels = images.to(device), labels.to(device)
    
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    #backward pass
    loss.backward()

    #step with optimizer
    optimizer.step()

    return loss

def test(model, test_loader):
    model.eval()

    with torch.no_grad():
        correct, total = 0, 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            oututs = model(images)
            _, predicated = torch.max(oututs.data, 1)
            total += labels.size(0)

            correct += (predicated == labels).sum().item()

        return correct/total

In [ ]:
nparameters = sum(p.numel() for p in resnet_.parameters())
nparameters

In [ ]:
model_pipeline()